<div style="padding: 35px;color:white;margin:10;font-size:200%;text-align:center;display:fill;border-radius:10px;overflow:hidden;background-image: url(https://images.pexels.com/photos/7078619/pexels-photo-7078619.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=1)"><b><span style='color:black'><strong>EABL STOCK PRICE PREDICTION </strong></span></b> </div> 

### <b> <span style='color:#16C2D5'>|</span> Business Objectives</b>
1. Build a robust time series model leveraging market indicators to forecast future EABL stock prices. 
2. Investigate viability of investing in EABL stock prices. 
3. Build an anomally detection system to identify unusual or unexpected patterns in EABL stock prices. 

### <b> <span style='color:#16C2D5'>|</span> Importing Libraries</b>

In [1]:
import pandas as pd 
import numpy as np
 
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline 

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA

## <div style="padding: 20px;color:white;margin:10;font-size:90%;text-align:left;display:fill;border-radius:10px;overflow:hidden;background-image: url(https://w0.peakpx.com/wallpaper/957/661/HD-wallpaper-white-marble-white-stone-texture-marble-stone-background-white-stone.jpg)"><b><span style='color:black'> Data Understanding</span></b> </div>


In [14]:

data = pd.read_csv('Data/EABL-2006-2024_JAN_STOCKS.csv')
data.head()

,Date,Open,High,Low,Close,Average,Volume
0,1/31/2024,104.00,111.00,104.0,110.0,106.00,42000
1,1/30/2024,105.00,105.00,101.0,104.0,104.00,15600
2,1/29/2024,105.00,105.00,99.0,103.5,100.00,596100
3,1/26/2024,116.25,116.25,100.0,100.0,104.50,60500
4,1/25/2024,119.75,120.00,118.0,118.0,118.25,5700


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4319 entries, 0 to 4318
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Date     4319 non-null   object 
 1   Open     4319 non-null   float64
 2   High     4319 non-null   float64
 3   Low      4319 non-null   float64
 4   Close    4319 non-null   float64
 5   Average  4319 non-null   float64
 6   Volume   4319 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 236.3+ KB


In [16]:
data.describe()

,Open,High,Low,Close,Average,Volume
count,4319.000000,4319.000000,4319.000000,4319.000000,4319.000000,4.319000e+03
mean,203.957513,206.961797,202.311125,204.684418,205.088562,2.415015e+05
std,59.289678,59.471967,58.546396,59.231241,59.292983,3.768558e+05
min,0.000000,98.000000,93.500000,96.500000,97.500000,0.000000e+00
25%,156.000000,158.250000,155.000000,156.000000,156.000000,2.785000e+04
50%,187.000000,190.000000,185.000000,188.000000,188.000000,1.122000e+05
75%,250.000000,250.500000,247.000000,250.000000,250.000000,3.055000e+05
max,425.000000,426.000000,416.000000,420.000000,424.000000,5.916300e+06


In [17]:
data.shape

(4319, 7)

## <div style="padding: 20px;color:white;margin:10;font-size:90%;text-align:left;display:fill;border-radius:10px;overflow:hidden;background-image: url(https://w0.peakpx.com/wallpaper/957/661/HD-wallpaper-white-marble-white-stone-texture-marble-stone-background-white-stone.jpg)"><b><span style='color:black'> 2. Data Cleaning and preparation</span></b> </div>


In [18]:
data_eda = data.copy()
data_eda   

,Date,Open,High,Low,Close,Average,Volume
0,1/31/2024,104.00,111.00,104.0,110.0,106.00,42000
1,1/30/2024,105.00,105.00,101.0,104.0,104.00,15600
2,1/29/2024,105.00,105.00,99.0,103.5,100.00,596100
3,1/26/2024,116.25,116.25,100.0,100.0,104.50,60500
4,1/25/2024,119.75,120.00,118.0,118.0,118.25,5700
...,...,...,...,...,...,...,...
4314,9/15/2006,141.00,142.00,140.0,141.0,141.00,226100
4315,9/14/2006,140.00,143.00,140.0,140.0,141.00,108500
4316,9/13/2006,141.00,144.00,140.0,142.0,141.00,190000
4317,9/12/2006,141.00,143.00,140.0,140.0,140.00,123900


In [19]:
# data2 = data_eda['Volume']=='23,40'
# index_with_value_2340 = data_eda.index[data2]

# # Print the index
# print(index_with_value_2340)

In [20]:
data_eda['Date'] = pd.to_datetime(data_eda['Date'])
data_eda['Volume'] = data_eda['Volume'].astype(int)